In [ ]:
# install Bonito last version and Pydrive
# if the cell is run and a numpy warning pops up, restart kernel and run again the cell
# source: https://github.com/nanoporetech/bonito/blob/v0.4.0/notebooks/bonito-train.ipynb

!pip install -q ont-bonito
!pip install -U -q PyDrive
!pip install -q tensorly
!pip install -q tensorly-torch
!pip install -q fast-ctc-decode

import os
import sys
import time
import random
from datetime import datetime
from itertools import starmap
from time import perf_counter
from functools import partial
import numpy as np
import pandas as pd
import toml
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.nn import Module, ModuleList, Sequential, Conv1d, BatchNorm1d, Dropout, ReLU, SiLU
from torch.nn.functional import ctc_loss, log_softmax
from torch.optim import AdamW
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR

from google.colab import auth
from google.colab import drive as gdrive
from oauth2client.client import GoogleCredentials
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

from bonito.nn import Permute
from bonito.util import accuracy, decode_ref, permute, concat
from bonito.data import ChunkDataSet
from fast_ctc_decode import beam_search, viterbi_search

# Tensor decomposition packages
import tensorly
from tltorch import FactorizedConv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.9/140.9 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 55.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 87.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 81.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 60.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.8/95.8 kB 12.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━

# Models

In [ ]:
class Model(Module):
    """
    Model template for QuartzNet style architectures

    https://arxiv.org/pdf/1910.10261.pdf
    """

    def __init__(self, config):
        super(Model, self).__init__()
        if 'qscore' not in config:
            self.qbias = 0.0
            self.qscale = 1.0
        else:
            self.qbias = config['qscore']['bias']
            self.qscale = config['qscore']['scale']

        self.config = config
        self.stride = config['block'][0]['stride'][0]
        self.alphabet = config['labels']['labels']
        self.features = config['block'][-1]['filters']
        self.encoder = Encoder(config)
        self.decoder = Decoder(self.features, len(self.alphabet))

    def forward(self, x):
        encoded = self.encoder(x)
        return self.decoder(encoded)

    def decode(self, x, beamsize=5, threshold=1e-3, qscores=False, return_path=False):
        x = x.exp().cpu().numpy().astype(np.float32)
        if beamsize == 1 or qscores:
            seq, path  = viterbi_search(x, self.alphabet, qscores, self.qscale, self.qbias)
        else:
            seq, path = beam_search(x, self.alphabet, beamsize, threshold)
        if return_path: return seq, path
        return seq


class Encoder(Module):
    """
    Builds the model encoder
    """
    def __init__(self, config):
        super(Encoder, self).__init__()
        self.config = config

        self.activations = {"relu": ReLU,"swish": SiLU}
        features = self.config['input']['features']
        activation = self.activations[self.config['encoder']['activation']]()
        encoder_layers = []

        for layer in self.config['block']:
            encoder_layers.append(
                Block(
                    features, layer['filters'], activation,
                    repeat=layer['repeat'], kernel_size=layer['kernel'],
                    stride=layer['stride'], dilation=layer['dilation'],
                    dropout=layer['dropout'], residual=layer['residual'],
                    separable=layer['separable'],
                )
            )

            features = layer['filters']

        self.encoder = Sequential(*encoder_layers)

    def forward(self, x):
        return self.encoder(x)


class TCSConv1d(Module):
    """
    Time-Channel Separable 1D Convolution
    """
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=False, separable=False):

        super(TCSConv1d, self).__init__()
        self.separable = separable

        if separable:
            # This layer cannot be factorised until "groups is implemented in tensorly-torch".
            self.depthwise = Conv1d(
                in_channels, in_channels, kernel_size=kernel_size, stride=stride,
                padding=padding, dilation=dilation, bias=bias, groups=in_channels
            )

            self.pointwise = Conv1d(
                in_channels, out_channels, kernel_size=1, stride=1,
                dilation=dilation, bias=bias, padding=0
            )

        else:
            self.conv = Conv1d(
                in_channels, out_channels, kernel_size=kernel_size,
                stride=stride, padding=padding, dilation=dilation, bias=bias
            )

    def forward(self, x):
        if self.separable:
            x = self.depthwise(x)
            x = self.pointwise(x)
        else:
            x = self.conv(x)
        return x


class Block(Module):
    """
    TCSConv, Batch Normalisation, Activation, Dropout
    """
    def __init__(self, in_channels, out_channels, activation, repeat=5, kernel_size=1, stride=1, dilation=1, dropout=0.0, residual=False, separable=False):

        super(Block, self).__init__()

        self.use_res = residual
        self.conv = ModuleList()

        _in_channels = in_channels
        padding = self.get_padding(kernel_size[0], stride[0], dilation[0])

        # add the first n - 1 convolutions + activation
        for _ in range(repeat - 1):
            self.conv.extend(
                self.get_tcs(
                    _in_channels, out_channels, kernel_size=kernel_size,
                    stride=stride, dilation=dilation,
                    padding=padding, separable=separable
                )
            )

            self.conv.extend(self.get_activation(activation, dropout))
            _in_channels = out_channels

        # add the last conv and batch norm
        self.conv.extend(
            self.get_tcs(
                _in_channels, out_channels,
                kernel_size=kernel_size,
                stride=stride, dilation=dilation,
                padding=padding, separable=separable
            )
        )

        # add the residual connection
        if self.use_res:
            self.residual = Sequential(*self.get_tcs(in_channels, out_channels))

        # add the activation and dropout
        self.activation = Sequential(*self.get_activation(activation, dropout))

    def get_activation(self, activation, dropout):
        return activation, Dropout(p=dropout)

    def get_padding(self, kernel_size, stride, dilation):
        if stride > 1 and dilation > 1:
            raise ValueError("Dilation and stride can not both be greater than 1")
        return (kernel_size // 2) * dilation

    def get_tcs(self, in_channels, out_channels, kernel_size=1, stride=1, dilation=1, padding=0, bias=False, separable=False):
        return [
            TCSConv1d(
                in_channels, out_channels, kernel_size,
                stride=stride, dilation=dilation, padding=padding,
                bias=bias, separable=separable
            ),
            BatchNorm1d(out_channels, eps=1e-3, momentum=0.1)
        ]

    def forward(self, x):
        _x = x
        for layer in self.conv:
            _x = layer(_x)
        if self.use_res:
            _x = _x + self.residual(x)
        return self.activation(_x)


class Decoder(Module):
    """
    Decoder
    """
    def __init__(self, features, classes):
        super(Decoder, self).__init__()
        self.layers = Sequential(
            Conv1d(features, classes, kernel_size=1, bias=True),
            Permute([2, 0, 1])
        )

    def forward(self, x):
        return log_softmax(self.layers(x), dim=2)

---
# DOWNLOAD SAMPLE DATASET

In [ ]:
chunks_link = "https://drive.google.com/open?id=1aciNfQs53eFRwnMggInY-Uisi-owtmzY" #@param {type:"string"}
references_link = "https://drive.google.com/open?id=1kcs_hZMndUIDX2n8dTxGrAgCvt_TpUcH" #@param {type:"string"}
reference_lengths_link = "https://drive.google.com/open?id=1-r7XymddP_3gKFb-7ohB_t14u7u4SGLm" #@param {type:"string"}


In [ ]:
# Authenticate and create PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# helper functions for importing data
def download_npy_from_link(fn, link):
    _, id = link.split('=')
    downloaded = drive.CreateFile({'id':id})
    downloaded.GetContentFile(fn)
    return np.load(fn)

print('Loading chunks.')
full_chunks = download_npy_from_link('chunks.npy',
                                chunks_link)
# Sections of squiggle that correspond with the target reference sequence
# Variable length and zero padded (upto 4096 samples).
# shape (1000000, 4096)
# dtype('float32')

print('Loading references.')
full_targets = download_npy_from_link('references.npy',
                                 references_link)
# Integer encoded target sequence {'A': 1, 'C': 2, 'G': 3, 'T': 4}
# Variable length and zero padded (default range between 128 and 256).
# shape (1000000, 256)
# dtype('uint8')

print('Loading reference lengths.')
full_target_lengths = download_npy_from_link('reference_lengths.npy',
                                        reference_lengths_link)
# Lengths of target sequences in references.npy
# shape (1000000,)
# dtype('uint8')


KeyboardInterrupt: ignored

# DOWNLOAD CONFIG AND LOAD STATE DICT

In [ ]:
# Authenticate and create PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

def download_toml_from_link(fn, link):
    _, id = link.split('=')
    downloaded = drive.CreateFile({'id':id})
    downloaded.GetContentFile(fn)
    return toml.load(fn)

# Load model configuration

# dna_r9.4.1@v1 is heavier
quartznet_config_link = "https://drive.google.com/open?id=1hKKE2Fzp3jdNyZI2h8jnOuwxBOvXWjp6"
quartznet_config = download_toml_from_link("dna_r9.4.1@v1.toml",quartznet_config_link)

# dna_r9.4.1@v2 is lighter
#quartznet_config_link = "https://drive.google.com/open?id=1IRDMrnE0WWeiRoioX7NHM5TXezl2jMkN"
#quartznet_config = download_toml_from_link("dna_r9.4.1@v2.toml",quartznet_config_link)

# The structure of the model is defined using a config file.
# This will make sense to those familar with QuartzNet

print('Loaded quartznet config.')


Loaded quartznet config.


# TEST SPLIT

In [ ]:
epochs = 1
random_seed = 25 #@param {type:"integer"}
batch_size = 16 #@param [2, 4, 8, 16, 28] {type:"raw"}
num_chunks = 10000 #@param [10, 100, 1000, 10000, 100000] {type:"raw"}
train_proportion = 0.80 #@param type:"slider", min:0.8, max:1000, step:1


In [ ]:
# Initialise random libs and setup cudnn
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)
torch.backends.cudnn.enabled = True
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# we exploit GPU for training
device = torch.device("cuda")

In [ ]:
# subset
chunks = full_chunks[:num_chunks]
targets = full_targets[:num_chunks]
target_lengths = full_target_lengths[:num_chunks]

# shuffle
shuf = np.random.permutation(chunks.shape[0])
chunks = chunks[shuf]
targets = targets[shuf]
target_lengths = target_lengths[shuf]

split = np.floor(chunks.shape[0] * train_proportion).astype(np.int32)

# Create dict - INITIAL ASSIGNATION LOOP


In [ ]:
# PARAMETERS:
version = "v1"
FACTORISATION = "tucker" # tucker // CP // TT
RANK = 1/10


In [ ]:

# mount users drive to load data
gdrive.mount('/content/drive', force_remount=True)

WEIGHT_NAME = f"weights_uncompressed_{version}"
WEIGHTS_PATH = f"/content/drive/MyDrive/Quartznet_weights/weights/compressed/"


model = Model(quartznet_config)
model.load_state_dict(torch.load(WEIGHTS_PATH + WEIGHT_NAME + ".pt"))


Mounted at /content/drive


<All keys matched successfully>

In [ ]:
# Initialise convolutional doctionary
conv_layers_dict = dict()

# Main loop
for enc_dec_layer in model._modules.keys():

  print("[OUTER LAYER]", enc_dec_layer)

  if enc_dec_layer == "encoder":
    for layer in model.encoder.encoder._modules.keys():
      for sublayer in model.encoder.encoder[int(layer)]._modules.keys():

        print("--------------------------------", layer, sublayer)

        if sublayer == "conv":
          # add to this condicion: and int(layer) <=3 (<3 works)

          for index in  model.encoder.encoder[int(layer)].conv._modules.keys():

            if type(model.encoder.encoder[int(layer)].conv[int(index)]) == TCSConv1d:
              print("[DEBUG ------->>]", model.encoder.encoder[int(layer)].conv[int(index)])

              if "conv" in model.encoder.encoder[int(layer)].conv[int(index)]._modules.keys():
                print(f"[DEBUG-PRE-conv] - layer: {layer} - sublayer: {sublayer} - index: {index} - current keys:", model.encoder.encoder[int(layer)].conv[int(index)]._modules.keys())

                # Add to the dict
                conv_layers_dict[f"conv_{layer}{index}"] = [layer, index, "conv"]
                print(f"[DEBUG-DICT] - conv_{layer}{index}")

                # Factorisation
                layer_to_factor = model.encoder.encoder[int(layer)].conv[int(index)].conv
                rank = layer_to_factor.weight.size(0)//2
                model.encoder.encoder[int(layer)].conv[int(index)].conv = FactorizedConv.from_conv(layer_to_factor, rank=RANK, factorization=FACTORISATION, decompose_weights=False)

                print("[DEBUG-POST]", type( model.encoder.encoder[int(layer)].conv[int(index)].conv))

              if "pointwise" in model.encoder.encoder[int(layer)].conv[int(index)]._modules.keys():
                print(f"[DEBUG-PRE-pointwise] - layer: {layer} - sublayer: {sublayer} - index: {index} - current keys:", model.encoder.encoder[int(layer)].conv[int(index)]._modules.keys())

                # Add to the dict
                conv_layers_dict[f"pointwise_{layer}{index}"] = [layer, index, "pointwise"]
                print(f"[DEBUG-DICT] - pointwise_{layer}{index}")

                # Factorisation
                layer_to_factor = model.encoder.encoder[int(layer)].conv[int(index)].pointwise
                rank = layer_to_factor.weight.size(0)//2
                model.encoder.encoder[int(layer)].conv[int(index)].pointwise = FactorizedConv.from_conv(layer_to_factor, rank=RANK, factorization=FACTORISATION, decompose_weights=False)

                print("[DEBUG]", type( model.encoder.encoder[int(layer)].conv[int(index)].pointwise))

              if "depthwise" in model.encoder.encoder[int(layer)].conv[int(index)]._modules.keys():
                print(f"[DEBUG-PRE-depthwise] - layer: {layer} - sublayer: {sublayer} - index: {index} - current keys:", model.encoder.encoder[int(layer)].conv[int(index)]._modules.keys())

                # Add to the dict
                conv_layers_dict[f"depthwise_{layer}{index}"] = [layer, index, "depthwise"]
                print(f"[DEBUG-DICT] - depthwise_{layer}{index}")

                # Factorisation not implemented due to "groups" parameter missing in tensorly-torch implementation of "from_conv()"
                """layer_to_factor = model.encoder.encoder[int(layer)].conv[int(index)].depthwise
                rank = layer_to_factor.weight.size(0)//2
                model.encoder.encoder[int(layer)].conv[int(index)].depthwise = FactorizedConv.from_conv(layer_to_factor, rank=RANK, factorization=FACTORISATION, decompose_weights=False)"""

                print("[DEBUG]", type( model.encoder.encoder[int(layer)].conv[int(index)].depthwise))

        elif sublayer == "residual":
          print("[DEBUG ------->>]", model.encoder.encoder[int(layer)].residual)
          """for index in  model.encoder.encoder[int(layer)].residual._modules.keys():

            if type(model.encoder.encoder[int(layer)].residual[int(index)]) == TCSConv1d:
              layer_to_factor = model.encoder.encoder[int(layer)].residual[int(index)].conv
              model.encoder.encoder[int(layer)].residual[int(index)].conv = FactorizedConv.from_conv(layer_to_factor, rank=RANK, factorization=FACTORISATION, decompose_weights=False)
              print("[DEBUG]", type(model.encoder.encoder[int(layer)].residual[int(index)].conv))"""

  elif enc_dec_layer == "decoder":
     # Only one layer, hardcoded
        """print("[DEBUG ------->>]", model.decoder.layers[0])
        layer_to_factor = model.decoder.layers[0]
        model.decoder.layers[0] = FactorizedConv.from_conv(layer_to_factor, rank=RANK, factorization=FACTORISATION, decompose_weights=False)
        print("[DEBUG]", type(model.decoder.layers[0]))
        """
        pass

In [ ]:
# visualise model architecture
model.eval()

In [ ]:
# mount users drive to save data

gdrive.mount('/content/drive', force_remount=True)
compressed_weights_savepath = '/content/drive/My Drive/Quartznet_weights/weights/compressed/' #@param {type:"string"}

# prevent overwriting of data
workdir = os.path.join(compressed_weights_savepath, f"weights_compressed_{version}_{FACTORISATION}_{RANK}.pt")

# Data to be written
torch.save(model.state_dict(), workdir)


Mounted at /content/drive


# TEST MODEL

In [ ]:
# Test function obtained from v3.0 (https://github.com/nanoporetech/bonito/blob/v0.3.0/bonito/training.py)

def ctc_label_smoothing_loss(log_probs, targets, lengths, weights):
    T, N, C = log_probs.shape
    log_probs_lengths = torch.full(size=(N, ), fill_value=T, dtype=torch.int64)
    loss = ctc_loss(log_probs.to(torch.float32), targets, log_probs_lengths, lengths, reduction='mean')
    label_smoothing_loss = -((log_probs * weights.to(log_probs.device)).mean())
    return {'loss': loss + label_smoothing_loss, 'ctc_loss': loss, 'label_smooth_loss': label_smoothing_loss}


def test(model, device, test_loader, min_coverage=0.5, criterion=None):

    if criterion is None:
        C = len(model.alphabet)
        weights = torch.cat([torch.tensor([0.4]), (0.1 / (C - 1)) * torch.ones(C - 1)]).to(device)
        criterion = partial(ctc_label_smoothing_loss, weights=weights)

    seqs = []
    model.eval()
    test_loss = 0
    accuracy_with_cov = lambda ref, seq: accuracy(ref, seq, min_coverage=min_coverage)

    with torch.no_grad():
        for batch_idx, (data, target, lengths) in enumerate(test_loader, start=1):
            log_probs = model(data.to(device))
            loss = criterion(log_probs, target.to(device), lengths.to(device))
            test_loss += loss['ctc_loss'] if isinstance(loss, dict) else loss
            seqs.extend([model.decode(p) for p in permute(log_probs, 'TNC', 'NTC')])

    refs = [
        decode_ref(target, model.alphabet) for target in test_loader.dataset.targets
    ]
    accuracies = [
        accuracy_with_cov(ref, seq) if len(seq) else 0. for ref, seq in zip(refs, seqs)
    ]

    mean = np.mean(accuracies)
    median = np.median(accuracies)
    return test_loss.item() / batch_idx, mean, median

In [ ]:
# savepath
workdir = '/content/drive/My Drive/Quartznet_weights/weights/compressed/' #@param {type:"string"}

# mount users drive to save data
gdrive.mount('/content/drive', force_remount=True)

# data generators
test_dataset = ChunkDataSet(chunks[split:], targets[split:], target_lengths[split:])

# data loader
test_loader = DataLoader(test_dataset, batch_size=batch_size,
                         num_workers=2, pin_memory=True)
# Cuda
model.to(device)

# 'Connectionist Temporal Classification' (CTC) loss fuction
# https://distill.pub/2017/ctc/
criterion = nn.CTCLoss(reduction='mean')

# report loss every
interval = 500 / num_chunks
log_interval = np.floor(len(test_dataset) / batch_size * interval)

exp_config = os.path.join(workdir, "experimental.log")
with open(exp_config, 'a') as c:
    c.write('Num training chunks: {}'.format(num_chunks) + '\n')
    c.write('random seed: {}'.format(random_seed) + '\n')
    c.write('epochs: {}'.format(epochs) + '\n')
    c.write('batch_size: {}'.format(batch_size) + '\n')
    c.write('train proportion: {}'.format(train_proportion) + '\n')

# DataFrame to store training logging information
training_results = pd.DataFrame()

for epoch in range(1, epochs + 1):
    # v3.0
    test_loss, mean, median = test(model, device, test_loader)

    # collate training and validation metrics
    epoch_result = pd.DataFrame(
        {'time':[datetime.today()],
         'epoch':[epoch],
         'validation_loss':[test_loss],
         'validation_mean':[mean],
         'validation_median':[median]})

    # update log file
    log_path = os.path.join(workdir, WEIGHTS_PATH + f"{RANK}_{FACTORISATION}_{version}.log")
    epoch_result.to_csv(log_path, mode='a', sep='\t', index=False)

    display(epoch_result)
    training_results = training_results.append(epoch_result)

display(training_results)

Mounted at /content/drive


,time,epoch,validation_loss,validation_mean,validation_median
0,2023-09-10 18:31:26.971587,1,7.390095,0.517643,0.0


<ipython-input-65-f694d721858c>:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  training_results = training_results.append(epoch_result)


,time,epoch,validation_loss,validation_mean,validation_median
0,2023-09-10 18:31:26.971587,1,7.390095,0.517643,0.0


# Results: parameters

Import example from:
https://github.com/JeanKossaifi/tensorly-notebooks/blob/master/05_pytorch_backend/cnn_acceleration_tensorly_and_pytorch.ipynb

In [ ]:
# ---------------0.13--------------------

# Tucker (rank=1/8)
# v1
# Original: 6678533
# Compressed: 1414436

# v2
# Original: 6649613
# Compressed: 1256840

# -------------------------------------

# CP (rank=1/8)
# v1
# Original: 6678533
# Compressed: 1393154

# v2
# Original: 6649613
# Compressed: 1238177

# -------------------------------------

# TT (rank=1/8)
# v1
# Original: 6678533
# Compressed: 1393357

# v2
# Original: 6649613
# Compressed: 1245262

# -------------------------------------


np.sum([np.prod(p.size()) for p in model.parameters()])

1245262

In [ ]:

# ---------------0.18--------------------

# Tucker (rank=1/6)
# v1
# Original: 6678533
# Compressed: 1686356

# v2
# Original: 6649613
# Compressed: 1542487
# -------------------------------------

# CP (rank=1/6)
# v1
# Original: 6678533
# Compressed: 1648459

# v2
# Original: 6649613
# Compressed: 1503985

# -------------------------------------

# TT (rank=1/6)
# v1
# Original: 6678533
# Compressed: 1646083

# v2
# Original: 6649613
# Compressed: 1496878

# -------------------------------------


np.sum([np.prod(p.size()) for p in model.parameters()])

1496878

In [ ]:

# ---------------0.35--------------------

# Tucker (rank=1/3)
# v1
# Original: 6678533
# Compressed: 2777034

# v2
# Original: 6649613
# Compressed: 2657400
# -------------------------------------

# CP (rank=1/3)
# v1
# Original: 6678533
# Compressed: 2651531

# v2
# Original: 6649613
# Compressed: 2532753

# -------------------------------------

# TT (rank=1/3)
# v1
# Original: 6678533
# Compressed: 2649484

# v2
# Original: 6649613
# Compressed: 2532887

# -------------------------------------


np.sum([np.prod(p.size()) for p in model.parameters()])

2532887

In [ ]:

# ---------------0.53--------------------

# Tucker (rank=1/2)
# v1
# Original: 6678533
# Compressed: 3907062

# v2
# Original: 6649613
# Compressed: 3829391
# -------------------------------------

# CP (rank=1/2)
# v1
# Original: 6678533
# Compressed: 3663306

# v2
# Original: 6649613
# Compressed: 3566653

# -------------------------------------

# TT (rank=1/2)
# v1
# Original: 6678533
# Compressed: 3659028

# v2
# Original: 6649613
# Compressed: 3555824

# -------------------------------------


np.sum([np.prod(p.size()) for p in model.parameters()])

3555824

In [ ]:

# ---------------0.7--------------------

# Tucker (rank=0.7)
# v1
# Original: 6678533
# Compressed: 5296541

# v2
# Original: 6649613
# Compressed: 5222229
# -------------------------------------

# CP (rank=0.7)
# v1
# Original: 6678533
# Compressed: 4866478

# v2
# Original: 6649613
# Compressed: 4802220

# -------------------------------------

# TT (rank=0.7)
# v1
# Original: 6678533
# Compressed: 4876031

# v2
# Original: 6649613
# Compressed: 4801893

# -------------------------------------


np.sum([np.prod(p.size()) for p in model.parameters()])

4801893

In [ ]:

# ---------------0.85--------------------

# Tucker (rank=0.8)
# v1
# Original: 6678533
# Compressed: 5973758

# v2
# Original: 6649613
# Compressed: 5922946
# -------------------------------------

# CP (rank=0.8)
# v1
# Original: 6678533
# Compressed: 5480530

# v2
# Original: 6649613
# Compressed: 5419782

# -------------------------------------

# TT (rank=0.8)
# v1
# Original: 6678533
# Compressed: 5468291

# v2
# Original: 6649613
# Compressed: 5416497

# -------------------------------------


np.sum([np.prod(p.size()) for p in model.parameters()])

5416497

# FINE TUNING MODEL

In [ ]:

model_savepath = '/content/drive/My Drive/Quartznet_weights/' #@param {type:"string"}
learning_rate = 0.001 #@param {type:"number"}
random_seed = 25 #@param {type:"integer"}
epochs = 20 #@param {type:"slider", min:1, max:1000, step:1}
batch_size = 16 #@param [2, 4, 8, 16, 28] {type:"raw"}
num_chunks = 10000 #@param [10, 100, 1000, 10000, 100000] {type:"raw"}
train_proportion = 0.80 #@param type:"slider", min:0.8, max:1000, step:1
dropout = 0.0 #@param {type:"slider", min:0.0, max:0.8}

In [ ]:
def train(model, device, train_loader, optimizer, use_amp=False, criterion=None, lr_scheduler=None, loss_log=None):

    if criterion is None:
        C = len(model.alphabet)
        weights = torch.cat([torch.tensor([0.4]), (0.1 / (C - 1)) * torch.ones(C - 1)]).to(device)
        criterion = partial(ctc_label_smoothing_loss, weights=weights)

    chunks = 0
    model.train()
    t0 = perf_counter()

    progress_bar = tqdm(
        total=len(train_loader), desc='[0/{}]'.format(len(train_loader.dataset)),
        ascii=True, leave=True, ncols=100, bar_format='{l_bar}{bar}| [{elapsed}{postfix}]'
    )
    smoothed_loss = {}

    with progress_bar:

        for data, targets, lengths in train_loader:

            optimizer.zero_grad()

            chunks += data.shape[0]

            # DEBUG MODE
            #display("[DEBUG]----------------", model(data.to(device)))

            log_probs = model(data.to(device))
            losses = criterion(log_probs, targets.to(device), lengths.to(device))

            if not isinstance(losses, dict):
                losses = {'loss': losses}

            if use_amp:
                pass
            else:
                losses['loss'].backward()

            optimizer.step()

            if lr_scheduler is not None: lr_scheduler.step()

            if not smoothed_loss:
                smoothed_loss = {k: v.item() for k,v in losses.items()}
            smoothed_loss = {k: 0.01 * v.item() + 0.99 * smoothed_loss[k] for k,v in losses.items()}

            progress_bar.set_postfix(loss='%.4f' % smoothed_loss['loss'])
            progress_bar.set_description("[{}/{}]".format(chunks, len(train_loader.dataset)))
            progress_bar.update()

            if loss_log is not None:
                loss_log.append({'chunks': chunks, 'time': perf_counter() - t0, **smoothed_loss})

    return smoothed_loss['loss'], perf_counter() - t0


In [ ]:
#@title Set experiment name
experiment_name = f'bonito_training_nodecodernoresidual_{FACTORISATION}_{RANK}_{version}' #@param {type:"string"}

# mount users drive to save data
gdrive.mount('/content/drive', force_remount=True)

# prevent overwriting of data
workdir = os.path.join(model_savepath, experiment_name)
if os.path.isdir(workdir):
    raise IOError('{} already exists. Select an alternative model_savepath.'.format(workdir))
os.makedirs(workdir)

# data generators
train_dataset = ChunkDataSet(chunks[:split], targets[:split], target_lengths[:split])
test_dataset = ChunkDataSet(chunks[split:], targets[split:], target_lengths[split:])


# data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size,
                          shuffle=True, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size,
                         num_workers=2, pin_memory=True)

# Cuda
model.to(device)

# load bonito model
model.train()

# 'Connectionist Temporal Classification' (CTC) loss fuction
# https://distill.pub/2017/ctc/
criterion = nn.CTCLoss(reduction='mean')

# set optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), amsgrad=True, lr=learning_rate)
schedular = CosineAnnealingLR(optimizer, epochs * len(train_loader))

# report loss every
interval = 500 / num_chunks
log_interval = np.floor(len(train_dataset) / batch_size * interval)

exp_config = os.path.join(workdir, "experimental.log")
with open(exp_config, 'a') as c:
    c.write('Num training chunks: {}'.format(num_chunks) + '\n')
    c.write('learning rate: {}'.format(learning_rate) + '\n')
    c.write('random seed: {}'.format(random_seed) + '\n')
    c.write('epochs: {}'.format(epochs) + '\n')
    c.write('batch_size: {}'.format(batch_size) + '\n')
    c.write('train proportion: {}'.format(train_proportion) + '\n')
    c.write('dropout: {}'.format(dropout) + '\n')

# DataFrame to store training logging information
training_results = pd.DataFrame()

for epoch in range(1, epochs + 1):
    # v3.0
    train_loss, duration = train(model, device, train_loader, optimizer)

    test_loss, mean, median = test(model, device, test_loader)

    # collate training and validation metrics
    epoch_result = pd.DataFrame(
        {'time':[datetime.today()],
         'duration':[int(duration)],
         'epoch':[epoch],
         'train_loss':[train_loss],
         'validation_loss':[test_loss],
         'validation_mean':[mean],
         'validation_median':[median]})

    # save model weights
    weights_path = os.path.join(workdir, "weights_%s.pt" % epoch)
    torch.save(model.state_dict(), weights_path)

    # update log file
    log_path = os.path.join(workdir, "training.log")
    epoch_result.to_csv(log_path, mode='a', sep='\t', index=False)

    display(epoch_result)
    training_results = training_results.append(epoch_result)

    schedular.step()

display(training_results)

Mounted at /content/drive


[1264/8000]:  16%|#########4                                                  | [01:32, loss=1.6007]

In [ ]:
np.sum([np.prod(p.size()) for p in model.parameters()])

# 1971831 -> 0.1 tucker v1
# 6678533 -> original

6678533